# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [8]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [9]:
# check the env directory
os.getenv('DFDATA')

'/gws/open/NYCOpenData/nycopendata/data'

In [12]:
# locate file in CUSP
API = r'erm2-nwe9/1473456214/erm2-nwe9.csv'

In [73]:
# Load dataset
data1 = pd.read_csv(os.getenv('DFDATA')+'/'+ API, usecols=[1,5,9,21,23,24])
data1.head()

,Created Date,Complaint Type,Incident Address,Resolution Description,Community Board,Borough
0,02/02/2015 02:15:00 PM,Water Conservation,166 ATKINS AVE,The Department of Environmental Protection sch...,Unspecified BROOKLYN,BROOKLYN
1,02/02/2015 02:16:04 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN
2,02/02/2015 02:17:59 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN
3,02/02/2015 05:11:34 PM,Illegal Parking,18-08 21 AVENUE,The Police Department responded to the complai...,01 QUEENS,QUEENS
4,02/02/2015 01:06:13 PM,Illegal Parking,1730 EAST 18 STREET,The Police Department responded to the complai...,15 BROOKLYN,BROOKLYN


# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [ ]:
# Check if all Boroughs and Community Districts are represented in the Data 

In [71]:
  # sort data ascending by value
data1 = data1.sort_values(by='Community Board')

In [72]:
data1.Borough.unique()

array(['Unspecified', 'BRONX', 'BROOKLYN', 'MANHATTAN', 'QUEENS',
       'STATEN ISLAND'], dtype=object)

In [29]:
data1['Community Board'].unique()

array(['0 Unspecified', '01 BRONX', '01 BROOKLYN', '01 MANHATTAN',
       '01 QUEENS', '01 STATEN ISLAND', '02 BRONX', '02 BROOKLYN',
       '02 MANHATTAN', '02 QUEENS', '02 STATEN ISLAND', '03 BRONX',
       '03 BROOKLYN', '03 MANHATTAN', '03 QUEENS', '03 STATEN ISLAND',
       '04 BRONX', '04 BROOKLYN', '04 MANHATTAN', '04 QUEENS', '05 BRONX',
       '05 BROOKLYN', '05 MANHATTAN', '05 QUEENS', '06 BRONX',
       '06 BROOKLYN', '06 MANHATTAN', '06 QUEENS', '07 BRONX',
       '07 BROOKLYN', '07 MANHATTAN', '07 QUEENS', '08 BRONX',
       '08 BROOKLYN', '08 MANHATTAN', '08 QUEENS', '09 BRONX',
       '09 BROOKLYN', '09 MANHATTAN', '09 QUEENS', '10 BRONX',
       '10 BROOKLYN', '10 MANHATTAN', '10 QUEENS', '11 BRONX',
       '11 BROOKLYN', '11 MANHATTAN', '11 QUEENS', '12 BRONX',
       '12 BROOKLYN', '12 MANHATTAN', '12 QUEENS', '13 BROOKLYN',
       '13 QUEENS', '14 BROOKLYN', '14 QUEENS', '15 BROOKLYN',
       '16 BROOKLYN', '17 BROOKLYN', '18 BROOKLYN', '26 BRONX', '27 BRONX',
      

All Boroughs and Community Districts are represented

In [51]:
# How many unique values do we have? 
print ('Number of Boroughs: %s'%(len(data1['Borough'].unique())))
print ('Number of Community Districts: %s'%(len(data1['Community Board'].unique())))

Number of Boroughs: 6
Number of Community Districts: 77


In [ ]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.

In [74]:
# Check for duplicates? Are these plausible?
print("Number of duplicated value: %s"%data1.duplicated().sum())

Number of duplicated value: 1205642


These duplicates are very likely to be plausible because I added 'created date' and 'incident address' to verify possible mistake when checking for duplicates. The possibility of a request that has the same address, created date and type of complaints is rather small. However, the number of missing values of 'Incident Address' is pretty high, we cannot rule out the possibility that duplicated values are identified because some values are missing, thus the duplicates might not be plausible and need double check by inviting other variables/columns from original dataset. 

In [75]:
# What about missing values? Can you detect any patterns? 
  # First check general missing values
data1.count()

Created Date              13588781
Complaint Type            13588781
Incident Address          10499543
Resolution Description    12996514
Community Board           13588781
Borough                   13588781
dtype: int64

In [76]:
borough = data1.Borough.unique()

In [79]:
 # Check for missing values in each borough
for i in borough:
    print('Borough: %s '%i)
    print(data1[data1['Borough'] == i].count())
    print('\n')

Borough: BROOKLYN 
Created Date              3841933
Complaint Type            3841933
Incident Address          3101933
Resolution Description    3675126
Community Board           3841933
Borough                   3841933
dtype: int64


Borough: MANHATTAN 
Created Date              2613583
Complaint Type            2613583
Incident Address          1973919
Resolution Description    2487255
Community Board           2613583
Borough                   2613583
dtype: int64


Borough: QUEENS 
Created Date              2934521
Complaint Type            2934521
Incident Address          2186308
Resolution Description    2787105
Community Board           2934521
Borough                   2934521
dtype: int64


Borough: STATEN ISLAND 
Created Date              639634
Complaint Type            639634
Incident Address          460144
Resolution Description    607607
Community Board           639634
Borough                   639634
dtype: int64


Borough: Unspecified 
Created Date              12

Each borough has missing values in 'Incident Address' and 'Resolution Description' parts. If we drop duplicated values before analyzing, the pattern will be similar

In [95]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?
data2 = data1[data1.duplicated() == False]

In [86]:
CT_replace = data2['Complaint Type'][data2['Resolution Description'].isnull() == True]

In [88]:
data2.fillna({'Resolution Description': CT_replace})

,Created Date,Complaint Type,Incident Address,Resolution Description,Community Board,Borough
0,02/02/2015 02:15:00 PM,Water Conservation,166 ATKINS AVE,The Department of Environmental Protection sch...,Unspecified BROOKLYN,BROOKLYN
1,02/02/2015 02:16:04 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN
2,02/02/2015 02:17:59 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN
3,02/02/2015 05:11:34 PM,Illegal Parking,18-08 21 AVENUE,The Police Department responded to the complai...,01 QUEENS,QUEENS
4,02/02/2015 01:06:13 PM,Illegal Parking,1730 EAST 18 STREET,The Police Department responded to the complai...,15 BROOKLYN,BROOKLYN
5,02/02/2015 04:43:29 PM,Illegal Parking,305 EAST 46 STREET,The Police Department responded to the complai...,06 MANHATTAN,MANHATTAN
6,02/02/2015 11:17:34 AM,Illegal Parking,13 DAVIS COURT,The Police Department responded to the complai...,01 STATEN ISLAND,STATEN ISLAND
7,02/02/2015 05:12:00 PM,Sewer,164 9 ST,The Department of Environmental Protection has...,Unspecified BROOKLYN,BROOKLYN
8,02/02/2015 09:12:38 PM,Illegal Parking,NaN,The Police Department responded and upon arriv...,11 QUEENS,QUEENS
9,02/02/2015 10:06:09 AM,Illegal Parking,75 MACDONOUGH STREET,The Police Department responded to the complai...,03 BROOKLYN,BROOKLYN


In [96]:
# Generate marker for unplausible Community Districts
# How do these districts look like? 
district = data2['Community Board']
data2['Community Index'] = district.str.extract('(\d+)', expand=True)
data2.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Created Date,Complaint Type,Incident Address,Resolution Description,Community Board,Borough,Community Index
0,02/02/2015 02:15:00 PM,Water Conservation,166 ATKINS AVE,The Department of Environmental Protection sch...,Unspecified BROOKLYN,BROOKLYN,NaN
1,02/02/2015 02:16:04 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN,11
2,02/02/2015 02:17:59 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN,11
3,02/02/2015 05:11:34 PM,Illegal Parking,18-08 21 AVENUE,The Police Department responded to the complai...,01 QUEENS,QUEENS,01
4,02/02/2015 01:06:13 PM,Illegal Parking,1730 EAST 18 STREET,The Police Department responded to the complai...,15 BROOKLYN,BROOKLYN,15


according to NYC COmmunity Portal https://www1.nyc.gov/site/planning/community/community-portal.page, the number of community district in each borough is: 
- Brooklyn: 18
- Manhattan: 12
- Bronx: 12
- Statan Island: 3
- Queens: 14

In [117]:
# Drop the marked districts
brooklyn =(data2['Community Index'] > 0) & (data2['Community Index'] < 19) & (data2['Borough'] == 'BROOKLYN')
manhattan =(data2['Community Index'] > 0) & (data2['Community Index'] < 13) & (data2['Borough'] == 'MANHATTAN')
bronx =(data2['Community Index'] > 0) & (data2['Community Index'] < 13) & (data2['Borough'] == 'BRONX')
statanisland =(data2['Community Index'] > 0) & (data2['Community Index'] < 4) & (data2['Borough'] == 'STATAN ISLAND')
queens =(data2['Community Index'] > 0) & (data2['Community Index'] < 15) & (data2['Borough'] == 'QUEENS')


In [113]:
communitydistrict = brooklyn | manhattan | bronx | statanisland | queens

In [114]:
data4 = data2[communitydistrict]
data4.head()

,Created Date,Complaint Type,Incident Address,Resolution Description,Community Board,Borough,Community Index
1,02/02/2015 02:16:04 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN,11.0
2,02/02/2015 02:17:59 AM,Noise - Commercial,2117 3 AVENUE,The Police Department responded to the complai...,11 MANHATTAN,MANHATTAN,11.0
3,02/02/2015 05:11:34 PM,Illegal Parking,18-08 21 AVENUE,The Police Department responded to the complai...,01 QUEENS,QUEENS,1.0
4,02/02/2015 01:06:13 PM,Illegal Parking,1730 EAST 18 STREET,The Police Department responded to the complai...,15 BROOKLYN,BROOKLYN,15.0
5,02/02/2015 04:43:29 PM,Illegal Parking,305 EAST 46 STREET,The Police Department responded to the complai...,06 MANHATTAN,MANHATTAN,6.0


In [125]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level
 # count number of complaints
data5 = data4.groupby('Community Board').count()
data5.head()

,Created Date,Complaint Type,Incident Address,Resolution Description,Borough,Community Index
Community Board,,,,,,
01 BRONX,94587,94587,81052,90073,94587,94587
01 BROOKLYN,230413,230413,181862,218015,230413,230413
01 MANHATTAN,89717,89717,56504,83595,89717,89717
01 QUEENS,216566,216566,168597,205134,216566,216566
02 BRONX,72168,72168,61010,68456,72168,72168


In [126]:
data5.reset_index(inplace=True)
data5.drop(['Created Date', 'Incident Address','Resolution Description', 'Borough','Community Index'], 
           axis=1, inplace=True)
data5['Rank'] = data5['Community Board'].rank(ascending=False)
data5.head()

,Community Board,Complaint Type,Rank
0,01 BRONX,94587,56.0
1,01 BROOKLYN,230413,55.0
2,01 MANHATTAN,89717,54.0
3,01 QUEENS,216566,53.0
4,02 BRONX,72168,52.0


In [127]:
# Safe reduced data frame (Community District level)
data5.sort(columns='Rank').head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,Community Board,Complaint Type,Rank
55,18 BROOKLYN,193135,1.0
54,17 BROOKLYN,223214,2.0
53,16 BROOKLYN,103416,3.0
52,15 BROOKLYN,173899,4.0
51,14 QUEENS,115551,5.0


# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [134]:
# Population by Community District
df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")
df_pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [133]:
# Check variables in file
len(df_pop.columns)

158

In [135]:
# How many community districts are in file? 
len(df_pop.cd_id.unique())

59

In [141]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311
# Save data frame
df_pop_2 = df_pop[['cd_id', 'Population Density (per sq. mile)']]
df_pop_2 = df_pop_2.sort_values(by='Population Density (per sq. mile)')
df_pop_2.head()

,cd_id,Population Density (per sq. mile)
25,SI02,6371.997113
24,SI03,6968.306388
58,QN14,9541.923526
49,QN13,9812.563966
26,SI01,12537.604960


In [142]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")
df_infr.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [143]:
# Check variables in file
len(df_infr.columns)

31

In [146]:
# How many community districts are in file? 
len(df_infr['Qualifying Name'].unique())

55

In [147]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data
df_infr.columns

Index([u'FIPS', u'Geographic Identifier', u'Qualifying Name', u'Households',
       u'Households: With An Internet Subscription',
       u'Households: Dial-Up Alone', u'Households: Dsl',
       u'Households: With Mobile Broadband',
       u'Households: Without Mobile Broadband', u'Households: Cable Modem',
       u'Households: With Mobile Broadband.1',
       u'Households: Without Mobile Broadband.1', u'Households: Fiber-Optic',
       u'Households: With Mobile Broadband.2',
       u'Households: Without Mobile Broadband.2',
       u'Households: Satellite Internet Service',
       u'Households: With Mobile Broadband.3',
       u'Households: Without Mobile Broadband.3',
       u'Households: Two or More Fixed Broadband Types, or Other',
       u'Households: With Mobile Broadband.4',
       u'Households: Without Mobile Broadband.4',
       u'Households: Mobile Broadband Alone or With Dialup',
       u'Households: Internet Access Without A Subscription',
       u'Households: No Internet Acc

In [ ]:
# Aggregate internet type by high and low connections

In [ ]:
# Save data frame 

# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [ ]:
# Harmonize identifier of dataframe 1


harmonize community districts into identifiers like BK01, BK02 similar to 'cd_id' in df_pop_2. 

In [ ]:
# Harmonize identifier of dataframe 2

In [ ]:
# Harmonize identifier of dataframe 3

In [ ]:
# Link the 3 dataframes
Linkedpd = pd.merge(data5, df_pop_2, on='cd_id', how = 'outer')
Linkedpd = pd.merge(Linkedpd, df_infr, on='cd_id', how = 'outer')

In [ ]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?